In [42]:
import os
import pandas as pd
import geopandas as gpd
import requests
import json
from ipyleaflet import Map, GeoJSON, Choropleth, Marker, MarkerCluster, Popup, GeoData
from ipywidgets import HTML

In [43]:
# Download the file from `url` and save it locally under `gtfs.zip`, then extract:
def gtfs_downloader(url):
    file_name = 'gtfs.zip'
    urllib.request.urlretrieve(url, file_name)
    import zipfile
    with zipfile.ZipFile(file_name,"r") as zip_ref:
        zip_ref.extractall("gtfs/")

In [44]:
#Give the URL of the TransitFeeds gtfs you want to analyze and create dataframes.
url = 'https://www.arcgis.com/sharing/rest/content/items/885399f83408473c8d815e40c5e702b7/data'
if (not os.path.isdir('./gtfs')):
    gtfs_downloader(url)

In [45]:
routes = pd.read_csv('gtfs/routes.txt')
routes.head()

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,8__151___,CRTM,151,MADRID (Plaza de Castilla) - ALCOBENDAS,NaN,3,http://www.crtm.es/tu-transporte-publico/autob...,8EBF42,FFFFFF
1,8__152_C__,CRTM,152C,MADRID (Plaza de Castilla)-S.S. DE LOS REYES (...,NaN,3,http://www.crtm.es/tu-transporte-publico/autob...,8EBF42,FFFFFF
2,8__153___,CRTM,153,MADRID (Plaza de Castilla)-ALCOBENDAS-ROSA LUX...,NaN,3,http://www.crtm.es/tu-transporte-publico/autob...,8EBF42,FFFFFF
3,8__154___,CRTM,154,MADRID (Chamartín)-S.S. REYES CIRCULAR (Por Fu...,NaN,3,http://www.crtm.es/tu-transporte-publico/autob...,8EBF42,FFFFFF
4,8__154_C__,CRTM,154C,MADRID (Plaza de Castilla)-S.S. DE LOS REYES (...,NaN,3,http://www.crtm.es/tu-transporte-publico/autob...,8EBF42,FFFFFF


In [46]:
stops = pd.read_csv('gtfs/stops.txt')
stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,par_8_06686,6686,AV.BURGOS-C.C.SANCHINARRO,Avda de Burgos 133,40.498047,-3.662996,A,http://www.crtm.es,0,NaN,Europe/Madrid,2
1,par_8_06687,6687,AV.BURGOS-DOMINICOS,Avda de Burgos 11300,40.501442,-3.659949,A,http://www.crtm.es,0,NaN,Europe/Madrid,2
2,par_8_06689,6689,CTRA.A1-CUESTA BLANCA,Ctra de Irún 13700,40.521770,-3.651011,B1,http://www.crtm.es,0,NaN,Europe/Madrid,2
3,par_8_06690,6690,CTRA.IRÚN-CONCESIONARIO,Ctra de Irún 14700,40.533794,-3.641858,B1,http://www.crtm.es,0,NaN,Europe/Madrid,2
4,par_8_06691,6691,AV.OLÍMPICA-C.C.LA VEGA,Avda Olímpica 9,40.535248,-3.636984,B1,http://www.crtm.es,0,NaN,Europe/Madrid,2


In [47]:
file_path = 'm8_ParadasPorItinerario.json'

if not os.path.exists(file_path):
  url = 'https://opendata.arcgis.com/datasets/46044e95c2f340e6a9e0790842bbbef2_3.csv?outSR=%7B%22latestWkid%22%3A25830%2C%22wkid%22%3A25830%7D'
  r = requests.get(url)
  with open(file_path, 'w') as f:
    f.write(r.content.decode("utf-8"))

with open(file_path, 'r') as f:
  ppi = pd.read_csv(f)

In [48]:
ppi.head()

,OBJECTID,MODO,CODIGOITINERARIO,CODIGOGESTIONLINEA,NUMEROLINEAUSUARIO,SENTIDO,TIPOITINERARIO,CODIGOESTACION,CODIGOPOSTE,CODIGOANDEN,...,IDFTRAMO,CODIGOOBSERVACION,CODIGOSUBLINEA,DENOMINACION_SAE,IDFLINEA,IDFITINERARIO,IDFESTACION,IDFPOSTE,IDFANDEN,CODIGOCTMESTACIONREDMETRO
0,4001,8,353151,8680,680,1,1,12511,*,NaN,...,8__680____-_1_IT_1_8_12511_*__TI_I_4,-,680101,DOCTOR VARELA-CANTERA,8__680___,8__680____680101_1_IT_1,8_12511,8_12511_*,8_12511_,NaN
1,4002,8,359409,8685,685,1,1,12511,*,NaN,...,8__685____-_1_IT_1_8_12511_*__TI_I_32,-,685101,DOCTOR VARELA-CANTERA,8__685___,8__685____685101_1_IT_1,8_12511,8_12511_*,8_12511_,NaN
2,4003,8,362303,8681,681,1,2,12511,*,NaN,...,8__681____va_1_IT_2_8_12511_*__TI_I_7,va,681104,DOCTOR VARELA-CANTERA,8__681___,8__681____681104_1_IT_2,8_12511,8_12511_*,8_12511_,NaN
3,4004,8,362182,8680,680,1,2,12511,*,NaN,...,8__680____i._1_IT_2_8_12511_*__TI_I_4,i.,680102,DOCTOR VARELA-CANTERA,8__680___,8__680____680102_1_IT_2,8_12511,8_12511_*,8_12511_,NaN
4,4005,8,364150,8682,682,1,2,12511,*,NaN,...,8__682____ma_1_IT_2_8_12511_*__TI_I_16,ma,682105,DOCTOR VARELA-CANTERA,8__682___,8__682____682105_1_IT_2,8_12511,8_12511_*,8_12511_,NaN


In [49]:
lines_from_moncloa = ppi.query("DENOMINACION_SAE == 'INTERCAMBIADOR DE MONCLOA' or DENOMINACION_SAE == 'INTERCAMBIADOR MONCLOA'")
lines_from_moncloa_unique_id = lines_from_moncloa['IDFLINEA'].unique()
lines_from_moncloa_unique_id

array(['8__625___', '8__628___', '8__631___', '8__621___', '8__632___',
       '8__629___', '8__627___', '8__661___', '8__664___', '8__622___',
       '8__645___', '8__661_A__', '8__662___', '8__623___', '8__624___',
       '8__635___', '8__656_A__', '8__656___', '8__651___', '8__652___',
       '8__611___', '8__683___', '8__687___', '8__688___', '8__672___',
       '8__612___', '8__686___', '8__681___', '8__686_A__', '8__643___',
       '8__671___', '8__672_A__', '8__653___', '8__641___', '8__642___',
       '8__611_A__', '8__691___', '8__657___', '8__655___', '8__613___',
       '8__673___', '8__684___', '8__654___', '8__682___', '8__657_A__',
       '8__658___', '8__601___', '8__659___'], dtype=object)

In [50]:
ppi[ppi['IDFLINEA'].isin(lines_from_moncloa_unique_id)]['IDFESTACION'].value_counts()

8_17480    154
8_06002    118
8_23       105
8_1333      96
8_06268     60
          ... 
8_10238      1
8_10739      1
8_10741      1
8_08784      1
8_09375      1
Name: IDFESTACION, Length: 1315, dtype: int64

In [51]:
lines_stop_23 = ppi.query("IDFESTACION == '8_23'")
lines_stop_23_unique_id = lines_stop_23['IDFLINEA'].unique()

In [52]:
ppi[ppi['IDFLINEA'].isin(lines_stop_23_unique_id)]['IDFESTACION'].value_counts()

8_17480    150
8_06002    113
8_23       110
8_1333     101
8_06268     63
          ... 
8_09959      1
8_10745      1
8_10743      1
8_08685      1
8_10742      1
Name: IDFESTACION, Length: 1301, dtype: int64

In [53]:
lines_pozuelo = ppi[ppi['DENOMINACION_SAE'].str.contains("POZUELO")==True]
lines_pozuelo_unique_id = lines_pozuelo['IDFLINEA'].unique()

In [54]:
lines_pozuelo['DENOMINACION_SAE'].value_counts()

CTRA.POZUELO-INS.NACIONAL DE SANIDAD    55
CTRA.POZUELO-URB.PINAR PLANTÍO          29
CTRA.POZUELO-RESIDENCIA                 29
CTRA.POZUELO-CIUDAD DEPORTIVA           27
CTRA.POZUELO-URB.MONTECLARO             18
CTRA.POZUELO-CENTRO DE PREVENCIÓN       18
CTRA.POZUELO-URB.EL PAULAR              16
CTRA.M502-EST.POZUELO OESTE             16
CTRA.POZUELO-AV. REY JUAN CARLOS I      12
AV.ESPAÑA-CTRA.POZUELO                   9
CTRA.POZUELO-FRANCISCO UMBRAL            9
AV.SAN JUAN DE DIOS-EST.CIEMPOZUELOS     8
CTRA.POZUELO-INSTITUTO                   8
SARASATE-CTRA.POZUELO                    4
PABLO PICASSO-CTRA.POZUELO               4
ARROYO POZUELO-GTA.RIO ZANCARA           4
ARROYO DE POZUELO-HÚMERA                 2
CUESTAS-CIEMPOZUELOS                     2
ARROYO POZUELO-GTA.SIERRA PARAMERA       2
ARROYO POZUELO-HINOJO                    2
ARROYO POZUELO-ALCORNOQUE                2
ANTONIO DÍAZ-EST.POZUELO                 2
ARROYO POZUELO-ANIS                      2
ARROYO POZU

In [55]:
ppi[ppi['IDFLINEA'].isin(lines_pozuelo_unique_id)]['IDFLINEA'].value_counts()

8__626___     590
8__567___     324
8__654___     258
8__561___     213
8__685___     197
8__653___     162
8__633___     153
8__561_B__    130
8__655___     120
8_N_901___    113
8__415___     112
8__562___     111
8_N_906___    103
8_N_902___     93
8__650___      92
8__658___      90
8__561_A__     89
8__416___      83
8__410___      76
8__563___      72
8__560___      63
8__659___      16
Name: IDFLINEA, dtype: int64

In [56]:
selected_lines_full = ['8_N_901___',
                  '8_N_906___',
                  '8_N_902___',
                  '8__650___',
                  '8__658___',
                  '8__563___',
                  '8__560___',
                  '8__656___',
                  '8__656_A__',
                  '8__657___',
                  '8__659___']
selected_lines = ['8__658___',
                  '8__656___',
                  '8__656_A__',
                  '8__563___',
                  '8__657___',
                  '8__659___']
ppi[ppi['IDFLINEA'].isin(selected_lines)]['IDFESTACION'].value_counts()

8_06002    12
8_10585    11
8_3444     11
8_06445     9
8_06478     9
           ..
8_08683     1
8_17964     1
8_08790     1
8_15359     1
8_06446     1
Name: IDFESTACION, Length: 285, dtype: int64

In [57]:
unique_selected_stops = ppi[ppi['IDFLINEA'].isin(selected_lines)]['IDFESTACION'].unique()
with open('selected_stops', 'w') as f:
    f.write('\n'.join(unique_selected_stops))

In [58]:
file_path = 'm8_estaciones.geojson'

if not os.path.exists(file_path):
  url = 'https://opendata.arcgis.com/datasets/19884a02ac044270b91fa478d80f7858_0.geojson'
  r = requests.get(url)
  with open(file_path, 'w') as f:
    f.write(r.content.decode("utf-8"))

#with open(file_path, 'r') as f:
stations = gpd.read_file(file_path)

In [59]:
selected_stations = stations[stations['IDESTACION'].isin(unique_selected_stops)]
len(selected_stations)

285

In [60]:
# Create a map
m = Map(center=(40.416775, -3.703790), zoom=10)

#geo_json = MarkerCluster(GeoJSON(data=data, style = {'color': 'green', 'opacity':1, 'weight':1.9, 'dashArray':'9', 'fillOpacity':0.1}))
#m.add_layer(geo_json)

geo_stops = GeoData(geo_dataframe = selected_stations)

m.add_layer(geo_stops)

# Display the map (if you dare)
m

Map(center=[40.416775, -3.70379], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …